In [1]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [3]:
# Define a function to create a model
def create_model(input_shape=(128, 128, 3), num_classes=19):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.5),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
# Set up the data generator (original dataset)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
train_data = datagen.flow_from_directory(
    '/Users/aranyabasu/crop_diseases(BORO_DATA)/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 10752 images belonging to 19 classes.


In [6]:
validation_generator = datagen.flow_from_directory(
    '/Users/aranyabasu/crop_diseases(BORO_DATA)/valid',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 2764 images belonging to 19 classes.


In [7]:
# Helper function to generate bootstrap samples (bags)
def create_bagged_generator(generator, n_samples):
    # Get all data from the generator
    images, labels = next(generator)  # Load the first batch
    for i in range(len(generator) - 1):  # Load the rest of the batches
        img_batch, lbl_batch = next(generator)
        images = np.vstack((images, img_batch))
        labels = np.vstack((labels, lbl_batch))

    # Randomly sample indices with replacement
    sampled_indices = np.random.choice(np.arange(len(images)), size=n_samples, replace=True)
    bagged_images = images[sampled_indices]
    bagged_labels = labels[sampled_indices]

    return ImageDataGenerator().flow(bagged_images, bagged_labels, batch_size=32)


In [8]:
# Train multiple models (bagging with repetition)
n_estimators = 5
models = []
model_dir = 'saved_models'
os.makedirs(model_dir, exist_ok=True)

n_samples = train_data.n  # Same size as the original training set

In [9]:
n_samples = train_data.n  # Same size as the original training set

for i in range(n_estimators):
    print(f"Training model {i + 1}/{n_estimators}")
    
    # Create a bagged generator
    bagged_train_generator = create_bagged_generator(train_data, n_samples)

    # Create and train model
    model = create_model()
    model.fit(
        bagged_train_generator,
        steps_per_epoch=len(bagged_train_generator),
        epochs=10,
        validation_data=validation_generator,
        verbose=1
    )
    
    # Save the model after training
    model_path = os.path.join(model_dir, f'model_{i+1}.h5')
    model_path = os.path.join(model_dir, f'model_{i+1}.keras')
    model.save(model_path)
    models.append(model_path)  # Save the model file paths


Training model 1/5
Epoch 1/10


2024-09-07 00:28:20.408337: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


336/336 [==============================] - 114s 337ms/step - loss: 3.4812 - accuracy: 0.1364 - val_loss: 2.0246 - val_accuracy: 0.3878
Epoch 2/10
336/336 [==============================] - 116s 346ms/step - loss: 1.8868 - accuracy: 0.4012 - val_loss: 1.2517 - val_accuracy: 0.5944
Epoch 3/10
336/336 [==============================] - 120s 356ms/step - loss: 1.3797 - accuracy: 0.5460 - val_loss: 1.0534 - val_accuracy: 0.6335
Epoch 4/10
336/336 [==============================] - 118s 353ms/step - loss: 1.1461 - accuracy: 0.6160 - val_loss: 0.9438 - val_accuracy: 0.6747
Epoch 5/10
336/336 [==============================] - 117s 350ms/step - loss: 0.9988 - accuracy: 0.6562 - val_loss: 0.8601 - val_accuracy: 0.7044
Epoch 6/10
336/336 [==============================] - 117s 350ms/step - loss: 0.8774 - accuracy: 0.6837 - val_loss: 0.8517 - val_accuracy: 0.7127
Epoch 7/10
336/336 [==============================] - 118s 352ms/step - loss: 0.7971 - accuracy: 0.7168 - val_loss: 0.8016 - val_accura

In [10]:
# Average predictions across all models
def predict_with_bagging(model_paths, data_generator):
    predictions = []
    
    for model_path in model_paths:
        model = tf.keras.models.load_model(model_path)
        predictions.append(model.predict(data_generator))
        
    avg_predictions = np.mean(predictions, axis=0)  # Averaging predictions
    return np.argmax(avg_predictions, axis=1)

# Evaluate on validation data
y_true = validation_generator.classes
y_pred = predict_with_bagging(models, validation_generator)

87/87 [==============================] - 50s 576ms/step


In [11]:
# Calculate accuracy
accuracy = np.mean(y_true == y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.7666


In [ ]:
model.save('Trained_model_V1.keras')